<a href="https://colab.research.google.com/github/itspritish/nlp/blob/master/NLP_Preproc_Util.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import re
import csv
import codecs
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation,Lambda
from keras.layers.merge import concatenate
from keras.models import Model,Sequential
from sklearn.model_selection import train_test_split 

In [0]:
data = pd.read_csv('drive/My Drive/dataml/quora_duplicate_questions.csv')
data = data.applymap(str)


In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [5]:
data.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [0]:
X1 = data['question1'].tolist()
X = data['question1'].tolist()
X2 = data['question2'].tolist()
X.extend(data['question2'].tolist())


y = data['is_duplicate'].tolist()

X1_train, X1_test, X2_train, X2_test, y_train, y_test = train_test_split(X1, X2, y, test_size=0.33)

In [46]:
"""tokenizer = Tokenizer(nb_words=1000)
texts ={}
try:
  texts = tokenizer.fit_on_texts(X)
  #sequences = tokenizer.texts_to_sequences(X)
  print(sequences)
except Exception as e:
  print('Exception is', e)  """


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/text.py:178: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


{}


In [0]:
texts

In [0]:
import operator 
def check_coverage(vocab,embeddings_index):
    a = {}
    oov = {}
    k = 0
    i = 0
    for word in tqdm(vocab):
        try:
            a[word] = embeddings_index[word]
            k += vocab[word]
        except:

            oov[word] = vocab[word]
            i += vocab[word]
            pass

    print('Found embeddings for {:.2%} of vocab'.format(len(a) / len(vocab)))
    print('Found embeddings for  {:.2%} of all text'.format(k / (k + i)))
    sorted_x = sorted(oov.items(), key=operator.itemgetter(1))[::-1]

    return sorted_x

In [0]:
from tqdm import tqdm
def build_vocab(sentences, verbose =  True):
    """
    :param sentences: list of list of words
    :return: dictionary of words and their count
    """
    vocab = {}
    for sentence in tqdm(sentences, disable = (not verbose)):
        for word in sentence.split():
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

In [0]:
vocab = build_vocab(X)
word2index, embedding_matrix = load_glove_embeddings('drive/My Drive/dataml/glove.6B.50d.txt', embedding_dim=50)
oov = check_coverage(vocab,word2index)

In [93]:
oov[500:600]

[('Cyrus', 459),
 ('C++?', 459),
 ('videos?', 458),
 ('note?', 458),
 ('it,', 457),
 ("haven't", 457),
 ('Hill-station', 456),
 ('thing?', 455),
 ('not,', 455),
 ('data?', 454),
 ('parents?', 454),
 ('State', 454),
 ('College', 454),
 ('OS', 452),
 ('IQ?', 452),
 ('end?', 451),
 ('beginners?', 450),
 ("what's", 450),
 ('email?', 449),
 ('management?', 446),
 ('Lord', 446),
 ('RBI', 446),
 ('3G', 444),
 ('Somme,', 442),
 ('Hitler', 441),
 ('WhatsApp?', 441),
 ('Kashmir', 441),
 ('power?', 439),
 ('life,', 436),
 ('degree?', 436),
 ('investment?', 433),
 ('Bill', 433),
 ('League', 432),
 ('Russian', 431),
 ('International', 431),
 ('jobs?', 430),
 ('1?', 430),
 ('cancer?', 430),
 ('PAN', 429),
 ('created?', 429),
 ('War?', 429),
 ('WiFi', 428),
 ('Islam?', 428),
 ('Kerala?', 428),
 ('works?', 427),
 ('accounts?', 427),
 ('Islam', 427),
 ('Chennai?', 426),
 ('first?', 426),
 ('Minister', 425),
 ('society?', 424),
 ('Khan', 424),
 ('song?', 423),
 ('Korean', 422),
 ('Wars', 422),
 ('age?',

In [0]:
import numpy as np
"""
https://gist.github.com/jovianlin/load_glove_embeddings.py
"""

def load_glove_embeddings(fp, embedding_dim, include_empty_char=True):
    """
    Loads pre-trained word embeddings (GloVe embeddings)
        Inputs: - fp: filepath of pre-trained glove embeddings
                - embedding_dim: dimension of each vector embedding
                - generate_matrix: whether to generate an embedding matrix
        Outputs:
                - word2coefs: Dictionary. Word to its corresponding coefficients
                - word2index: Dictionary. Word to word-index
                - embedding_matrix: Embedding matrix for Keras Embedding layer
    """
    # First, build the "word2coefs" and "word2index"
    word2coefs = {} # word to its corresponding coefficients
    word2index = {} # word to word-index
    with open(fp) as f:
        for idx, line in enumerate(f):
            try:
                data = [x.strip().lower() for x in line.split()]
                word = data[0]
                coefs = np.asarray(data[1:embedding_dim+1], dtype='float32')
                word2coefs[word] = coefs
                if word not in word2index:
                    word2index[word] = len(word2index)
            except Exception as e:
                print('Exception occurred in `load_glove_embeddings`:', e)
                continue
        # End of for loop.
    # End of with open
    if include_empty_char:
        word2index[''] = len(word2index)
    # Second, build the "embedding_matrix"
    # Words not found in embedding index will be all-zeros. Hence, the "+1".
    vocab_size = len(word2coefs)+1 if include_empty_char else len(word2coefs)
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    for word, idx in word2index.items():
        embedding_vec = word2coefs.get(word)
        if embedding_vec is not None and embedding_vec.shape[0]==embedding_dim:
            embedding_matrix[idx] = np.asarray(embedding_vec)
    # return word2coefs, word2index, embedding_matrix
    return word2index, np.asarray(embedding_matrix)